---

# Coopang 광고 데이터 처리

1. `from requests import get ` 로 처리 안됨, selenium 으로 다이내믹 처리


## html parsing

In [ ]:
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()

browser = webdriver.Chrome(options=options)

base_url = "https://www.coupang.com/np/search?component"
search_term = "썬크림"

browser.get(f"{base_url}=&q={search_term}&channel=user")


print(browser.page_source) # html source code 추출

In [ ]:
# --- bs4 처리 

soup = BeautifulSoup(browser.page_source, "html.parser")

search_product_list = soup.find("ul", class_="search-product-list")

# ul 태그 내부, li 태그 (1 level)
products = search_product_list.find_all('li', recursive=False)


# 페이지 총 제품 의 개수 36
print(len(products))
print()

# product
for product in products:
    print(product)
    print()

- 이전 프로젝트에서 중간에 아예 다른 태그 가 있었는데, 지금은 같은 li태그중에 이름만 다른경우 같다.

```
# li 태그 중 class 네임에 따라 출력, # find, none 논리로 출력 

cnt = 0
for product in products:
    
    if product.find("li", class_="__ad-badge"):
        print('ad')

    
```

---

## ad 제거

- html source 코드 확인 후, ad 제품 'search-product__ad-badge' 필터링


In [1]:
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()

browser = webdriver.Chrome(options=options)

base_url = "https://www.coupang.com/np/search?component"
search_term = "썬크림"


browser.get(f"{base_url}=&q={search_term}&channel=user")

soup = BeautifulSoup(browser.page_source, "html.parser")

search_product_list = soup.find("ul", class_="search-product-list")

filtered_products = [ product for product in search_product_list.find_all('li', recursive=False) if 'search-product__ad-badge' not in product['class'] ]


In [2]:
len(filtered_products)

27

In [3]:
# 36개중 9개 광고 제거 데이터 출력

for product in filtered_products:
    print(product)


<li class="search-product" data-coupon-nudge-text="" data-coupon-tag-area="true" data-is-rocket="true" data-product-id="4583248357" data-vendor-item-id="4498803481" data-winner-vendor-item-id="4498803481" id="4583248357" style="height: auto;"><a class="search-product-link" data-is-soldout="" data-item-id="5620780687" data-product-id="4583248357" data-srp-log='{"group":"PRODUCT", "itemId":"5620780687", "productId":"4583248357", "vendorItemId":"4498803481", "page":"1", "listSize":"36", "isCcidEligible":false, "displayCcidBadge":false, "wowOnlyInstantDiscountRate": -1, "snsDiscountRate" : -1, "isLoyaltyMember": false }' data-vendor-item-id="4498803481" href="/vp/products/4583248357?itemId=5620780687&amp;vendorItemId=4498803481&amp;q=%EC%8D%AC%ED%81%AC%EB%A6%BC&amp;itemsCount=36&amp;searchId=5c088b1d5cff448eb352727dbcf6b6fe&amp;rank=1" style="height: 506px;" target="_blank"><dl class="search-product-wrap" style="height: 506px;"><dt class="image"> <img alt="본트리 베리 에센스 선블럭 SPF50+ PA++++, 50m

---

## 데이터 추출


1. 이미지

2. 브랜드

3. 네임

4. 가격

5. 와우회원 추가 할인 쿠폰 적용 가격

6. 링크



In [4]:
# 제품 링크

anchor = filtered_products[0].find('a')

print('https://www.coupang.com/'+anchor['href'])

https://www.coupang.com//vp/products/4583248357?itemId=5620780687&vendorItemId=4498803481&q=%EC%8D%AC%ED%81%AC%EB%A6%BC&itemsCount=36&searchId=5c088b1d5cff448eb352727dbcf6b6fe&rank=1


In [5]:
link = 'https://www.coupang.com/'+anchor['href']

In [6]:
print(link)

https://www.coupang.com//vp/products/4583248357?itemId=5620780687&vendorItemId=4498803481&q=%EC%8D%AC%ED%81%AC%EB%A6%BC&itemsCount=36&searchId=5c088b1d5cff448eb352727dbcf6b6fe&rank=1


---

In [7]:
# 브랜드, 제품명, SPF, PA

filtered_products[0].select_one('.name')

<div class="name">본트리 베리 에센스 선블럭 SPF50+ PA++++, 50ml, 2개</div>

In [8]:
s = filtered_products[0].select_one('.name').string

In [9]:
split_result = s.split(' ', 1)  # 첫 번째 공백을 기준으로 문자열 분리

brand_name = split_result[0]
product_name = split_result[1]

In [10]:
brand_name,product_name

('본트리', '베리 에센스 선블럭 SPF50+ PA++++, 50ml, 2개')

---

In [11]:
# 가격

filtered_products[0].select_one('.price-value')

<strong class="price-value">15,000</strong>

In [12]:
price = filtered_products[0].select_one('.price-value').string

In [13]:
price

'15,000'

In [14]:
type(price)

bs4.element.NavigableString

In [16]:
p = int(filtered_products[0].select_one('.price-value').string.replace(",", ""))

In [17]:
p

15000

In [18]:
type(p)

int

---

In [19]:
# 와우 회원 할인 쿠폰 유무  추가 할인 쿠폰 여부 있으면 if true, 없으면, None

l = filtered_products[0].select_one('.coupon-wot-nudge-ticket_left_border_amount')
    

In [20]:
print(l)

<span class="coupon-wot-nudge-ticket_left_border_amount">1,600</span>


---

In [21]:
# 유닛 별 가격

filtered_products[0].select_one('.unit-price')

<span class="unit-price">(<em>10</em>ml당 <em>1,500</em>원) </span>

---

https://thumbnail7.coupangcdn.com/thumbnails/remote/230x230ex/image/retail/images/aa87de05-2343-4704-ab69-946e854d13a43873205657323349374.png

In [22]:
# 이미지

img_link = 'https:'+filtered_products[1].select_one('.search-product-wrap-img')['src']
print(img_link)

https://thumbnail7.coupangcdn.com/thumbnails/remote/230x230ex/image/retail/images/4583290751164806-872ed467-d261-408d-8269-6a6f5e189e66.jpg


<img src = https://thumbnail9.coupangcdn.com/thumbnails/remote/230x230ex/image/retail/images/13745872385703788-0927705e-e248-4364-aa3d-bd0b4bde068a.jpg>

---

In [23]:
# rank, 10위까지만 표시 됨
# 데이터 처리순 랭크, 

num = filtered_products[7].find('span', class_='number')
num

<span class="number no-8">8 </span>

---

## 데이터 추출 후 딕셔너리 형태로 처리, list화

- result
- 검색어 input 처리

In [ ]:
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()

browser = webdriver.Chrome(options=options)

base_url = "https://www.coupang.com/np/search?component"


search_term = input()


browser.get(f"{base_url}=&q={search_term}&channel=user")

soup = BeautifulSoup(browser.page_source, "html.parser")

search_product_list = soup.find("ul", class_="search-product-list")

filtered_products = [ product for product in search_product_list.find_all('li', recursive=False) if 'search-product__ad-badge' not in product['class'] ]


results = []

for i in range(len(filtered_products)):
    
    # brand_name, product_name
    s = filtered_products[i].select_one('.name').string
    split_result = s.split(' ', 1)  # 첫 번째 공백을 기준으로 문자열 분리

    brand_name = split_result[0]
    product_name = split_result[1]
    
    # link
    anchor = filtered_products[i].find('a')

    product_data = {
        
        'img_link' : 'https:'+filtered_products[i].select_one('.search-product-wrap-img')['src'],
        'brand' : brand_name,
        'name' : product_name,
        'price': filtered_products[i].select_one('.price-value').string,
        'link' : 'https://www.coupang.com/'+anchor['href']
    }

    results.append(product_data)

In [ ]:
results

In [ ]:
len(results)

---

## 함수화

- 데이터 정제 
    - 쉼표 처리,
    - Price int 처리

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


def remove_ad_coopang(keyword):

    options = Options()
    browser = webdriver.Chrome(options=options)

    base_url = "https://www.coupang.com/np/search?component"
    search_term = keyword

    browser.get(f"{base_url}=&q={search_term}&channel=user")

    soup = BeautifulSoup(browser.page_source, "html.parser")

    search_product_list = soup.find("ul", class_="search-product-list")

    filtered_products = [ product for product in search_product_list.find_all('li', recursive=False) if 'search-product__ad-badge' not in product['class'] ]


    results = []

    for i in range(len(filtered_products)):

        # brand_name, product_name
        s = filtered_products[i].select_one('.name').string
        split_result = s.split(' ', 1)  # 첫 번째 공백을 기준으로 문자열 분리

        brand_name = split_result[0]
        product_name = split_result[1]

        # link
        anchor = filtered_products[i].find('a')

        product_data = {

            'img_link' : 'https:'+filtered_products[i].select_one('.search-product-wrap-img')['src'],
            'brand' : brand_name,
            'name' : product_name.replace(",", ""),
            'price': int(filtered_products[i].select_one('.price-value').string.replace(",", "")),
            'link' : 'https://www.coupang.com/'+anchor['href']
        }

        results.append(product_data)
    
    return results

In [ ]:
remove_ad_coopang('썬크림')

---

## CSV 파일 처리

In [ ]:
keyword = input('what do you want to search for? ')

products = remove_ad_coopang(keyword)

# 입력 받은 키워드 이름으로 파일 오픈, 없으면 새로 생성
file = open(f"{keyword}.csv", "w",encoding="utf-8-sig")

# 파일에 헤더 입력 , \n
file.write("Image,Brand,Name,Price,URL\n")

# jobs 리스트, job dict
for product in products:
    file.write(f"{product['img_link']},{product['brand']},{product['name']},{product['price']},{product['link']}\n")

In [ ]:
file.close()

---